In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/OH_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1198187110,OH,"1901 E 13th St, CLEVELAND, OH",41.501934,-81.682772,3.903511e+14
1,1200627808,OH,"4235 Secor Road, Toledo, OH",41.695649,-83.624250,3.909501e+14
2,1243727106,OH,"390 W. Nationwide Blvd, COLUMBUS, OH",39.968125,-83.012576,3.904900e+14
3,1385077107,OH,"555 Edgar Waldo Way 0.0, Columbus, OH",39.972951,-83.017489,3.904900e+14
4,1470677104,OH,"21000 Aerospace Pkwy, CLEVELAND, OH",41.396808,-81.863242,3.903598e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1198187110,OH,"1901 E 13th St, CLEVELAND, OH",41.501934,-81.682772,3.903511e+14
1,1200627808,OH,"4235 Secor Road, Toledo, OH",41.695649,-83.624250,3.909501e+14
2,1243727106,OH,"390 W. Nationwide Blvd, COLUMBUS, OH",39.968125,-83.012576,3.904900e+14
3,1385077107,OH,"555 Edgar Waldo Way 0.0, Columbus, OH",39.972951,-83.017489,3.904900e+14
4,1470677104,OH,"21000 Aerospace Pkwy, CLEVELAND, OH",41.396808,-81.863242,3.903598e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6661,3467957103,OH,"400 Lakemont Park Blvd, HILLIARD, OH",40.470975,-78.395592,NaN
7642,4957177004,OH,"935 hermitage, MEDINA, OH",41.899533,-87.670867,NaN
10872,5099767110,OH,"4594 US HIGHWAY 30, CRESTLINE, OH",41.126715,-85.197995,NaN
11691,9778267102,OH,"One MISSION PARK, GROVEPORT, OH",39.855788,-82.888569,NaN
12718,2189347810,OH,"3518 US HIGHWAY 52, RIPLEY, OH",33.968158,-79.748421,NaN
16089,5699088401,OH,"4819 us route 62, Millersburg, OH",36.375115,-92.312496,NaN
17225,4100458503,OH,3769 Kingman Hill Dr Ste 220 3769 Columbus Pik...,35.189443,-114.053006,NaN
17601,3902407108,OH,"27767 GILCHRIST DR, EUCLID, OH",41.628109,-81.493373,NaN
18236,6952597001,OH,"2478 CANYON CREEK DR, HINCKLEY, OH",41.267644,-81.775156,NaN
23611,6362577801,OH,"8686 BUNCH FLOWER CT, WESTERVILLE, OH",40.161240,-82.879667,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('39')] #OH
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/15 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

15


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13233/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6661,3467957103,OH,"400 Lakemont Park Blvd, HILLIARD, OH",40.470975,-78.395592,[]
7642,4957177004,OH,"935 hermitage, MEDINA, OH",41.899533,-87.670867,[]
10872,5099767110,OH,"4594 US HIGHWAY 30, CRESTLINE, OH",41.126715,-85.197995,[]
11691,9778267102,OH,"One MISSION PARK, GROVEPORT, OH",39.855788,-82.888569,[390490094401003]
12718,2189347810,OH,"3518 US HIGHWAY 52, RIPLEY, OH",33.968158,-79.748421,[]
16089,5699088401,OH,"4819 us route 62, Millersburg, OH",36.375115,-92.312496,[]
17225,4100458503,OH,3769 Kingman Hill Dr Ste 220 3769 Columbus Pik...,35.189443,-114.053006,[]
17601,3902407108,OH,"27767 GILCHRIST DR, EUCLID, OH",41.628109,-81.493373,[390351523012002]
18236,6952597001,OH,"2478 CANYON CREEK DR, HINCKLEY, OH",41.267644,-81.775156,[391034001001008]
23611,6362577801,OH,"8686 BUNCH FLOWER CT, WESTERVILLE, OH",40.161240,-82.879667,[390410117401017]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_13233/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6661,3467957103,OH,"400 Lakemont Park Blvd, HILLIARD, OH",40.470975,-78.395592,NaN
7642,4957177004,OH,"935 hermitage, MEDINA, OH",41.899533,-87.670867,NaN
10872,5099767110,OH,"4594 US HIGHWAY 30, CRESTLINE, OH",41.126715,-85.197995,NaN
11691,9778267102,OH,"One MISSION PARK, GROVEPORT, OH",39.855788,-82.888569,390490094401003
12718,2189347810,OH,"3518 US HIGHWAY 52, RIPLEY, OH",33.968158,-79.748421,NaN
16089,5699088401,OH,"4819 us route 62, Millersburg, OH",36.375115,-92.312496,NaN
17225,4100458503,OH,3769 Kingman Hill Dr Ste 220 3769 Columbus Pik...,35.189443,-114.053006,NaN
17601,3902407108,OH,"27767 GILCHRIST DR, EUCLID, OH",41.628109,-81.493373,390351523012002
18236,6952597001,OH,"2478 CANYON CREEK DR, HINCKLEY, OH",41.267644,-81.775156,391034001001008
23611,6362577801,OH,"8686 BUNCH FLOWER CT, WESTERVILLE, OH",40.161240,-82.879667,390410117401017


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6661,3467957103,OH,"400 Lakemont Park Blvd, HILLIARD, OH",40.470975,-78.395592,NaN
7642,4957177004,OH,"935 hermitage, MEDINA, OH",41.899533,-87.670867,NaN
10872,5099767110,OH,"4594 US HIGHWAY 30, CRESTLINE, OH",41.126715,-85.197995,NaN
12718,2189347810,OH,"3518 US HIGHWAY 52, RIPLEY, OH",33.968158,-79.748421,NaN
16089,5699088401,OH,"4819 us route 62, Millersburg, OH",36.375115,-92.312496,NaN
17225,4100458503,OH,3769 Kingman Hill Dr Ste 220 3769 Columbus Pik...,35.189443,-114.053006,NaN
28278,2438668701,OH,"15920 State Route 62, Damascus, OH",36.393952,-92.082233,NaN
30428,1265847805,OH,"615 HARBOR AVE, Toledo, OH",41.919900,-83.371802,NaN
31337,4924757408,OH,"6205 N SUMMIT ST, TOLEDO, OH",41.732376,-83.480608,NaN
32221,3506697210,OH,"10001 N Stateline Road, Union City, OH",40.208821,-84.805750,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
6661,3467957103,NaN
7642,4957177004,NaN
10872,5099767110,NaN
11691,9778267102,390490094401003
12718,2189347810,NaN
16089,5699088401,NaN
17225,4100458503,NaN
17601,3902407108,390351523012002
18236,6952597001,391034001001008
23611,6362577801,390410117401017


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/OH_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,677402,1198187110,OH,"1901 E 13th St, CLEVELAND, OH",41.501934,-81.682772,3.903511e+14
1,677403,1200627808,OH,"4235 Secor Road, Toledo, OH",41.695649,-83.624250,3.909501e+14
2,677404,1243727106,OH,"390 W. Nationwide Blvd, COLUMBUS, OH",39.968125,-83.012576,3.904900e+14
3,677405,1385077107,OH,"555 Edgar Waldo Way 0.0, Columbus, OH",39.972951,-83.017489,3.904900e+14
4,677406,1470677104,OH,"21000 Aerospace Pkwy, CLEVELAND, OH",41.396808,-81.863242,3.903598e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,677402,1198187110,OH,"1901 E 13th St, CLEVELAND, OH",41.501934,-81.682772,3.903511e+14,NaN
1,677403,1200627808,OH,"4235 Secor Road, Toledo, OH",41.695649,-83.624250,3.909501e+14,NaN
2,677404,1243727106,OH,"390 W. Nationwide Blvd, COLUMBUS, OH",39.968125,-83.012576,3.904900e+14,NaN
3,677405,1385077107,OH,"555 Edgar Waldo Way 0.0, Columbus, OH",39.972951,-83.017489,3.904900e+14,NaN
4,677406,1470677104,OH,"21000 Aerospace Pkwy, CLEVELAND, OH",41.396808,-81.863242,3.903598e+14,NaN
...,...,...,...,...,...,...,...,...
32634,710036,9615497009,OH,"5221 South Cleveland Massillon Rd, BARBERTON, OH",40.996192,-81.637160,3.915353e+14,NaN
32635,710037,9729447700,OH,"4100 REGENT ST STE 4C, columbus, OH",40.052713,-82.915985,3.904901e+14,NaN
32636,710038,9924358503,OH,"100 Lock St # 45215, Cincinnati, OH",39.225109,-84.454267,3.906103e+14,NaN
32637,710039,9968218302,OH,"1177 Farnsworth Rd, Waterville, OH",41.497553,-83.735146,3.909501e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,677402,1198187110,OH,"1901 E 13th St, CLEVELAND, OH",41.501934,-81.682772,3.903511e+14,NaN,390351078023020.0
1,677403,1200627808,OH,"4235 Secor Road, Toledo, OH",41.695649,-83.624250,3.909501e+14,NaN,390950078021004.0
2,677404,1243727106,OH,"390 W. Nationwide Blvd, COLUMBUS, OH",39.968125,-83.012576,3.904900e+14,NaN,390490032002035.0
3,677405,1385077107,OH,"555 Edgar Waldo Way 0.0, Columbus, OH",39.972951,-83.017489,3.904900e+14,NaN,390490032002008.0
4,677406,1470677104,OH,"21000 Aerospace Pkwy, CLEVELAND, OH",41.396808,-81.863242,3.903598e+14,NaN,390359805001013.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1198187110,OH,"1901 E 13th St, CLEVELAND, OH",41.501934,-81.682772,390351078023020.0
1,1200627808,OH,"4235 Secor Road, Toledo, OH",41.695649,-83.624250,390950078021004.0
2,1243727106,OH,"390 W. Nationwide Blvd, COLUMBUS, OH",39.968125,-83.012576,390490032002035.0
3,1385077107,OH,"555 Edgar Waldo Way 0.0, Columbus, OH",39.972951,-83.017489,390490032002008.0
4,1470677104,OH,"21000 Aerospace Pkwy, CLEVELAND, OH",41.396808,-81.863242,390359805001013.0
...,...,...,...,...,...,...
32634,9615497009,OH,"5221 South Cleveland Massillon Rd, BARBERTON, OH",40.996192,-81.637160,391535320012002.0
32635,9729447700,OH,"4100 REGENT ST STE 4C, columbus, OH",40.052713,-82.915985,390490101001037.0
32636,9924358503,OH,"100 Lock St # 45215, Cincinnati, OH",39.225109,-84.454267,390610274002002.0
32637,9968218302,OH,"1177 Farnsworth Rd, Waterville, OH",41.497553,-83.735146,390950089062025.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
6661,3467957103,OH,"400 Lakemont Park Blvd, HILLIARD, OH",40.470975,-78.395592,NaN
7642,4957177004,OH,"935 hermitage, MEDINA, OH",41.899533,-87.670867,NaN
10872,5099767110,OH,"4594 US HIGHWAY 30, CRESTLINE, OH",41.126715,-85.197995,NaN
12718,2189347810,OH,"3518 US HIGHWAY 52, RIPLEY, OH",33.968158,-79.748421,NaN
16089,5699088401,OH,"4819 us route 62, Millersburg, OH",36.375115,-92.312496,NaN
17225,4100458503,OH,3769 Kingman Hill Dr Ste 220 3769 Columbus Pik...,35.189443,-114.053006,NaN
28278,2438668701,OH,"15920 State Route 62, Damascus, OH",36.393952,-92.082233,NaN
30428,1265847805,OH,"615 HARBOR AVE, Toledo, OH",41.919900,-83.371802,NaN
31337,4924757408,OH,"6205 N SUMMIT ST, TOLEDO, OH",41.732376,-83.480608,NaN
32221,3506697210,OH,"10001 N Stateline Road, Union City, OH",40.208821,-84.805750,NaN


Convert to File

In [19]:
fips_OH = fips_merge_drop

In [20]:
fips_OH.to_csv("../../../data/state_data/geo/geo_fips/23/OH_fips_scraped.csv") 